In [1]:
import os
import pandas as pd
import json
import numpy as np
import pickle
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer
from datasets import load_from_disk, load_metric, Dataset, load_dataset

from captum.concept import TCAV
from captum.concept._utils.data_iterator import dataset_to_dataloader, CustomIterableDataset

In [ ]:
checkpoint = '/work3/s174498/finetuning-sentiment-model-test-head3/checkpoint-500'

model = RobertaForSequenceClassification.from_pretrained(checkpoint,output_hidden_states = True,return_dict = True,)
model.eval()

In [ ]:
model.roberta.encoder.layer[0].attention

In [ ]:
tcav = TCAV(model, layers=['roberta.encoder.layer[0]'])

In [ ]:
# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer_checkpoint(examples["sentence"], truncation=True)

tokenized_test = test_dataset.map(preprocess_function, batched=True)

## Hvordan bruger man den her : 

In [ ]:
def get_tensor_from_filename(filename):
    ds = torchtext.data.TabularDataset(path=filename,
                                       fields=[('text', torchtext.data.Field()),
                                               ('label', torchtext.data.Field())],
                                       format='csv')
    const_len = 7
    for concept in ds:
        concept.text = concept.text[:const_len]
        concept.text += ['pad'] * max(0, const_len - len(concept.text))
        text_indices = torch.tensor([TEXT.vocab.stoi[t] for t in concept.text], device=device)
        yield text_indices

In [ ]:
def get_tensor_from_filename(filename):
    img = Image.open(filename).convert("RGB")
    return transform(img)

## Hent concept data

In [ ]:
def assemble_concept(name, id, concepts_path="/work3/a174498/sst2_dataset/"):
    concept_path = os.path.join(concepts_path, name) + "/"
    dataset = CustomIterableDataset(get_tensor_from_filename, concept_path)
    concept_iter = dataset_to_dataloader(dataset)

    return Concept(id=id, name=name, data_iter=concept_iter)

In [ ]:
def assemble_concept(name, id, concepts_path="data/tcav/text-sensitivity"):
    dataset = CustomIterableDataset(get_tensor_from_filename, concepts_path)
    concept_iter = dataset_to_dataloader(dataset, batch_size=1)
    return Concept(id=id, name=name, data_iter=concept_iter)

In [ ]:
tcav.interpret(tokenized_test, experimental_sets=experimental_sets)